In [1]:
import sqlite3
from sqlite3 import Error

def select_all_tasks(conn, query1):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(query1) 
    rows = cur.fetchall()
    for row in rows:
        print(row)

In [2]:
database = "sqlite_db_pythonsqlite.db"
conn = sqlite3.connect(database)

##### Q1 : 
Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.


In [3]:
query = """
SELECT name
FROM Facilities
WHERE membercost > 0
"""
with conn :
    select_all_tasks(conn, query)

('Tennis Court 1',)
('Tennis Court 2',)
('Massage Room 1',)
('Massage Room 2',)
('Squash Court',)


#### Q2: 
How many facilities do not charge a fee to members?

In [4]:
query = """
SELECT COUNT(*)
FROM Facilities
WHERE membercost = 0
"""
with conn :
    select_all_tasks(conn, query)

(4,)


#### Q3: 
Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. 

In [5]:
query = """
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0 AND membercost< 0.2*monthlymaintenance
"""
with conn :
    select_all_tasks(conn, query)

(0, 'Tennis Court 1', 5, 200)
(1, 'Tennis Court 2', 5, 200)
(4, 'Massage Room 1', 9.9, 3000)
(5, 'Massage Room 2', 9.9, 3000)
(6, 'Squash Court', 3.5, 80)


#### Q4: 
Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [6]:
query = """
SELECT *
FROM Facilities
WHERE facid IN(1,5)
"""
with conn :
    select_all_tasks(conn, query)

(1, 'Tennis Court 2', 5, 25, 8000, 200)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)


#### Q5: 
Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [7]:
query = """
SELECT name, monthlymaintenance,
    CASE WHEN monthlymaintenance < 100 THEN 'cheap'
        ELSE 'expensive' END as cost_qual
FROM Facilities
"""
with conn :
    select_all_tasks(conn, query)

('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Badminton Court', 50, 'cheap')
('Table Tennis', 10, 'cheap')
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Squash Court', 80, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')


#### Q6: 
You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [8]:
query = """
SELECT surname, firstname
FROM Members
ORDER BY memID DESC
"""
with conn :
    cur = conn.cursor()
    cur.execute(query) 
    rows = cur.fetchall()
    for i in range(0,5):
        print(rows[i])

('Smith', 'Darren')
('Crumpet', 'Erica')
('Hunt', 'John')
('Tupperware', 'Hyacinth')
('Purview', 'Millicent')


##### Q7: 
Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [9]:
# Tennis court facid = 0, 1

query = """
SELECT DISTINCT f.name as fac_name, (m.firstname || ' ' || m.surname) as full_name
FROM Facilities as f
INNER JOIN Bookings as b
USING(facid)
INNER JOIN members as m
USING(memid)
WHERE f.facid IN(0,1)
ORDER BY full_name
"""
with conn :
    select_all_tasks(conn, query)

('Tennis Court 1', 'Anne Baker')
('Tennis Court 2', 'Anne Baker')
('Tennis Court 2', 'Burton Tracy')
('Tennis Court 1', 'Burton Tracy')
('Tennis Court 1', 'Charles Owen')
('Tennis Court 2', 'Charles Owen')
('Tennis Court 2', 'Darren Smith')
('Tennis Court 1', 'David Farrell')
('Tennis Court 2', 'David Farrell')
('Tennis Court 2', 'David Jones')
('Tennis Court 1', 'David Jones')
('Tennis Court 1', 'David Pinker')
('Tennis Court 1', 'Douglas Jones')
('Tennis Court 1', 'Erica Crumpet')
('Tennis Court 2', 'Florence Bader')
('Tennis Court 1', 'Florence Bader')
('Tennis Court 2', 'GUEST GUEST')
('Tennis Court 1', 'GUEST GUEST')
('Tennis Court 1', 'Gerald Butters')
('Tennis Court 2', 'Gerald Butters')
('Tennis Court 2', 'Henrietta Rumney')
('Tennis Court 1', 'Jack Smith')
('Tennis Court 2', 'Jack Smith')
('Tennis Court 1', 'Janice Joplette')
('Tennis Court 2', 'Janice Joplette')
('Tennis Court 2', 'Jemima Farrell')
('Tennis Court 1', 'Jemima Farrell')
('Tennis Court 1', 'Joan Coplin')
('Tenni

##### Q8: 
Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [10]:
# b.starttime LIKE '2012-09-14%'
# f.membercost > 30 OR (f.guestcost > 30 AND b.memid = 0)

query = """
SELECT f.name, (m.firstname || ' ' || m.surname) as full_name,
    CASE WHEN b.memid = 0 THEN f.guestcost*b.slots
        ELSE f.membercost*b.slots END as cost_booking
FROM Bookings as b
INNER JOIN Facilities as f
USING(facid)
INNER JOIN Members as m
USING(memid)
WHERE b.starttime LIKE '2012-09-14%' AND
    (f.membercost*b.slots > 30 OR (f.guestcost*b.slots > 30 AND b.memid = 0))
ORDER BY cost_booking DESC
"""
with conn :
    select_all_tasks(conn, query)

('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)


##### Q9: 
This time, produce the same result as in Q8, but using a subquery.

In [39]:
query = """

SELECT 
    f.name,
    (m.firstname || ' ' || m.surname) as full_name, 
    (f.guestcost * fac_slot.guest_slot + 
        f.membercost * fac_slot.mem_slot) as tot_cost
FROM (
    SELECT bookid, memid, facid, ifnull(mem_slot,0) as mem_slot, ifnull(guest_slot,0) as guest_slot
    FROM (
        WITH guest_b AS (
            SELECT bookid, memid, facid, slots
                FROM bookings
                WHERE memid = 0
                ),
        mem_b AS (
            SELECT bookid, memid, facid, slots
                FROM bookings
                WHERE memid != 0
                )
        SELECT b.bookid, b.memid, b.facid,
            mem_b.slots AS mem_slot,
            guest_b.slots AS guest_slot
        FROM bookings as b
        LEFT JOIN mem_b
        ON mem_b.bookid = b.bookid
        LEFT JOIN guest_b
        ON guest_b.bookid = b.bookid
        WHERE b.starttime LIKE '2012-09-14%')) as fac_slot
INNER JOIN members as m
USING(memid)
INNER JOIN facilities as f
USING(facid)    
WHERE (tot_cost > 30)
ORDER BY tot_cost DESC
"""


with conn :
    select_all_tasks(conn, query)

('Massage Room 2', 'GUEST GUEST', 320.0)
('Massage Room 1', 'GUEST GUEST', 160.0)
('Massage Room 1', 'GUEST GUEST', 160.0)
('Massage Room 1', 'GUEST GUEST', 160.0)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)


#### Q10: 
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [12]:
# Total Revenue : 
# Find count of **SLOTS** by guests / members (guests (memid=0), and members (else)), group by facid
# Multiply count of guests/members by cost

query = """
SELECT f.name,
    (f.guestcost * fac_slot.guest_slot_tot + 
        f.membercost * fac_slot.mem_slot_tot) as tot_rev
FROM  
    (WITH guest_b AS (
        SELECT bookid, facid, slots
            FROM bookings
            WHERE memid = 0
            ),
    mem_b AS (
        SELECT bookid, facid, slots
            FROM bookings
            WHERE memid != 0
            )
    SELECT b.facid,
        SUM(mem_b.slots) AS mem_slot_tot,
        SUM(guest_b.slots) AS guest_slot_tot
    FROM bookings as b
    LEFT JOIN mem_b
    ON mem_b.bookid = b.bookid
    LEFT JOIN guest_b
    ON guest_b.bookid = b.bookid
    GROUP BY b.facid) as fac_slot
INNER JOIN facilities as f
USING(facid)
WHERE tot_rev < 1000
ORDER BY tot_rev desc
"""



with conn :
    select_all_tasks(conn, query)

('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


#### Q11: 
Produce a report of members and who recommended them in alphabetic surname,firstname order


In [13]:
# get distinct recommendedby names
# WHERE IN(list distinct names)
query = """
SELECT surname || ', ' || firstname as name
FROM Members
WHERE memid IN(
    SELECT DISTINCT recommendedby
    FROM Members
    WHERE recommendedby NOT NULL)
"""

with conn :
    select_all_tasks(conn, query)


('Smith, Darren',)
('Smith, Tracy',)
('Rownam, Tim',)
('Joplette, Janice',)
('Butters, Gerald',)
('Tracy, Burton',)
('Stibbons, Ponder',)
('Jones, David',)
('Farrell, Jemima',)
('Bader, Florence',)
('Baker, Timothy',)
('Genting, Matthew',)
('Purview, Millicent',)


#### Q12: 
Find the facilities with their usage by member, but not guests 

In [14]:
# SUM(slots) 
# where memid != 0
# group by facid, member


query = """
SELECT facid, memid, SUM(slots)
FROM Bookings
WHERE memid !=0
GROUP BY facid, memid
"""

with conn :
    select_all_tasks(conn, query)

(0, 2, 93)
(0, 3, 18)
(0, 4, 57)
(0, 5, 171)
(0, 6, 93)
(0, 7, 81)
(0, 8, 12)
(0, 9, 3)
(0, 10, 51)
(0, 11, 84)
(0, 12, 18)
(0, 13, 3)
(0, 14, 75)
(0, 15, 3)
(0, 16, 45)
(0, 17, 51)
(0, 20, 3)
(0, 22, 21)
(0, 24, 15)
(0, 26, 27)
(0, 28, 18)
(0, 35, 12)
(0, 36, 3)
(1, 1, 57)
(1, 2, 6)
(1, 3, 18)
(1, 4, 24)
(1, 5, 9)
(1, 6, 9)
(1, 7, 33)
(1, 8, 168)
(1, 9, 96)
(1, 10, 141)
(1, 11, 99)
(1, 12, 114)
(1, 13, 3)
(1, 14, 3)
(1, 15, 24)
(1, 16, 21)
(1, 24, 36)
(1, 27, 3)
(1, 28, 3)
(1, 30, 3)
(1, 35, 12)
(2, 1, 432)
(2, 2, 102)
(2, 3, 12)
(2, 5, 63)
(2, 6, 6)
(2, 7, 30)
(2, 8, 36)
(2, 9, 48)
(2, 10, 18)
(2, 11, 24)
(2, 12, 30)
(2, 13, 21)
(2, 14, 36)
(2, 15, 27)
(2, 16, 21)
(2, 17, 21)
(2, 21, 96)
(2, 24, 21)
(2, 26, 6)
(2, 29, 15)
(2, 30, 6)
(2, 33, 3)
(2, 35, 6)
(2, 36, 6)
(3, 1, 56)
(3, 2, 56)
(3, 3, 150)
(3, 4, 18)
(3, 5, 2)
(3, 6, 48)
(3, 7, 10)
(3, 8, 8)
(3, 9, 6)
(3, 10, 50)
(3, 11, 22)
(3, 12, 2)
(3, 13, 24)
(3, 14, 10)
(3, 15, 86)
(3, 16, 48)
(3, 17, 34)
(3, 20, 54)
(3, 21, 32)
(3, 22

#### Q13: 
Find the facilities usage by month, but not guests

In [15]:
# GROUP BY EXTRACT(MONTH from starttime), facid

query = """
SELECT 
    strftime('%m', starttime) as month_group,
    facid, 
    SUM(slots) as usage
FROM Bookings
WHERE memid != 0
GROUP BY month_group, facid
"""

with conn :
    select_all_tasks(conn, query)

('07', 0, 201)
('07', 1, 123)
('07', 2, 165)
('07', 3, 98)
('07', 4, 166)
('07', 5, 8)
('07', 6, 50)
('07', 7, 140)
('07', 8, 110)
('08', 0, 339)
('08', 1, 345)
('08', 2, 414)
('08', 3, 296)
('08', 4, 316)
('08', 5, 18)
('08', 6, 184)
('08', 7, 316)
('08', 8, 303)
('09', 0, 417)
('09', 1, 414)
('09', 2, 507)
('09', 3, 400)
('09', 4, 402)
('09', 5, 28)
('09', 6, 184)
('09', 7, 404)
('09', 8, 443)
